<a href="https://colab.research.google.com/github/marcosfelt/interview_study_plan/blob/main/algorithms/dynamic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Programming

From this youtube course:  https://www.youtube.com/watch?v=oBt53YbR9Kk

In [2]:
from typing import List, Optional, Dict, Union

In [3]:
print_in = lambda args, fun: print(f"{args}: ", fun(*args))

## Fibonacci

The fibonacci series is a positive integer series where each element past the second is the sum of the previous two:

1, 1, 2, 3, 5, 8, 13, ...

Can we write an algorithm to caclulate the series for any $n$?

In [ ]:
# Slow implementation
def fib(n):
  if n<=2:
    return 1
  else:
    return fib(n-1) + fib(n-2)

For small $n$ the above implementation is fine:

In [ ]:
print("7:", fib(7))
print("8:", fib(8))
print("13:", fib(13))

7: 13
8: 21
13: 233


However, for slightly larger n, it will stall (try running below):

In [ ]:
print("50:", fib(50))

We can think of the call stack as below:

![](https://github.com/marcosfelt/interview_study_plan/blob/main/static/fibonacci_tree.png?raw=true)

Since each of the subtrees is repeated, we can _memoize_ or store the result of the calculation the first time we do it. Therefore, we transform the algorithm from $O(2^n)$ to $O(n)$.

In [ ]:
# Fast implementation
def fib_memo(n, memo=None):
  if memo is None:
    memo = {}
  if n<=2:
    return 1
  elif n in memo:
    return memo[n]
  else:
    r = fib_memo(n-1, memo) + fib_memo(n-2, memo)
    memo[n] = r
    return r

In [ ]:
print("50:", fib_memo(50))

50: 12586269025


Yay! We can now calculate the fibonacci series of large numbers in linear time.

## Grid Traveler

In [ ]:
def grid_traveler(m,n):
  # Base case
  if m==1 and n ==1:
    return 1
  # Trivial case
  if m==0 or n==0:
    return 0
  # Down + # Right
  return grid_traveler(m-1, n) + grid_traveler(m, n-1)

Works for small $n$

In [ ]:
print("1,1",grid_traveler(1,1))
print("2,3:",grid_traveler(2,3))


1,1 1
2,3: 3


Bigger $n$ is slower

In [ ]:
print("18,18:",grid_traveler(18,18))

We found in our analyssi that:
- We want to use memoization for subtrees
- We can leverage symmetry between rows and columns (since its down and right always). 

In [ ]:
def grid_traveler_memo(m,n, memo=None):
  if memo is None:
    memo = {}

  # Check if (m,n) in memo
  if (m,n) in memo:
    return memo[(m,n)]
  elif (n,m) in memo:
    return memo[(n,m)]

  # Base case
  if m==1 and n ==1:
    return 1
  # Trivial case
  elif m==0 or n==0:
    return 0
  # Down + Right
  else:
    count = grid_traveler_memo(m-1, n, memo) + grid_traveler_memo(m, n-1, memo)
    memo[(m,n)] = count
    return count

In [ ]:
print("18,18:",grid_traveler_memo(18,18))

18,18: 2333606220


## CanSum

Can you generate a target value from the sum of a  set of numbers? Return a boolean.

In [3]:
def can_sum(target_sum: int, numbers: List[int]):
  # Base case - at botom
  if target_sum == 0:
    return True

  # Can't add 
  if target_sum < 0:
    return False
    
  for num in numbers:
    remainder = target_sum - num
    if can_sum(remainder, numbers):
      return True
  return False

In [10]:
# These work great!
print_in((7, [2,3]), can_sum)
print_in((7, [5, 3, 4, 7]), can_sum)
print_in((7, [2,4]), can_sum)

(7, [2, 3]):  True
(7, [5, 3, 4, 7]):  True
(7, [2, 4]):  False


This is a large number that makes it reallly slow.

In [11]:
print_in((300, [7,14]), can_sum)

KeyboardInterrupt: ignored

This is because it has $O(n^m)$ time and $O(m)$ space complexity where m is the size of the target sum and n is the length of the numbers array.

In [16]:
def can_sum_memo(
    target_sum: int,
    numbers: List[int],
    memo: Optional[Dict[int, int]] = None
)-> bool:
  # Memoization
  if memo is None:
    memo = {}
  if target_sum in memo:
    return memo[target_sum]

  # Base case - at botom
  if target_sum == 0:
    return True

  # Can't add any more 
  if target_sum < 0:
    return False
    
  for num in numbers:
    remainder = target_sum - num
    if can_sum_memo(remainder, numbers, memo):
      memo[target_sum] = True
      return True
  
  memo[target_sum] = False
  return False

In [17]:
print_in((300, [7,14]), can_sum_memo)

(300, [7, 14]):  False


## HowSum

In [19]:
def how_sum(
    target_sum: int,
    numbers: List[int],
)-> Union[None, List]:
  if target_sum == 0:
    return []
  if target_sum < 0:
    return None
  
  for num in numbers:
    remainder = target_sum - num
    res = how_sum(remainder, numbers)
    if res is not None:
      return res + [num]
  
  return None

In [24]:
# These work great!
print_in((7, [2,3]), how_sum)
print_in((7, [5, 3, 4, 7]), how_sum)
print_in((7, [2,4]), how_sum)
print_in((8, [3,5,2]), how_sum)

(7, [2, 3]):  [3, 2, 2]
(7, [5, 3, 4, 7]):  [4, 3]
(7, [2, 4]):  None
(8, [3, 5, 2]):  [2, 3, 3]


This is realy slow!

In [25]:
print_in((300, [7,14]), how_sum)

KeyboardInterrupt: ignored

m = target_sum

Time complexity: $O(n^m+m)$ (+m is because of the copy array
Space complexity: $O(m)$

In [33]:
def how_sum_memoize(
    target_sum: int,
    numbers: List[int],
    memo: Optional[Dict[int, int]]=None
)-> Union[None, List]:
  if memo is None:
    memo = {}
  if target_sum in memo:
    return memo[target_sum]
  if target_sum == 0:
    return []
  if target_sum < 0:
    return None
  
  for num in numbers:
    remainder = target_sum - num
    res = how_sum_memoize(remainder, numbers, memo)
    if res is not None:
      memo[target_sum] = res + [num]
      return memo[target_sum]
  
  memo[target_sum] = None
  return None

In [42]:
print_in((300, [7,14]), how_sum_memoize)
print_in((300, [ 7, 25,6]), how_sum_memoize)

(300, [7, 14]):  None
(300, [7, 25, 6]):  [6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]


Time complexity: $O(n*m*m)=O(nm^2)$
Space complexity: Mostly from memo object $O(m*m)$ = $O(m^2)$

## bestSum

In [4]:
def best_sum(target_sum: int, numbers: List[int])->Union[List, None]:
  # Base cases
  if target_sum == 0:
    return []
  if target_sum < 0:
    return None

  best_arr = None
  for num in numbers:
    remainder = target_sum - num
    res = best_sum(remainder, numbers)
    if res is not None:
      arr = res + [num]
      if (best_arr is None) or (len(arr) < len(best_arr)) :
        best_arr = arr

  return best_arr


In [5]:
print_in((7, [5,3,4,6]), best_sum)
print_in((7, [5,3]), best_sum)

(7, [5, 3, 4, 6]):  [4, 3]
(7, [5, 3]):  None


This is really slow

In [ ]:
print_in((100, [1,2,5,25]), best_sum)

In [11]:
def best_sum_memo(
    target_sum: int, 
    numbers: List[int],
    memo: Optional[Dict[str, List[int]]]=None,
)->Union[List, None]:
  if memo is None:
    memo = {}
  if target_sum in memo:
    return memo[target_sum]
  # Base cases
  if target_sum == 0:
    return []
  if target_sum < 0:
    return None

  best_arr = None
  for num in numbers:
    remainder = target_sum - num
    res = best_sum_memo(remainder, numbers, memo)
    if res is not None:
      arr = res + [num]
      if (best_arr is None) or (len(arr) < len(best_arr)) :
        best_arr = arr

  memo[target_sum] = best_arr
  return best_arr

In [12]:
print_in((100, [1,2,5,25]), best_sum_memo)

(100, [1, 2, 5, 25]):  [25, 25, 25, 25]


## canConstruct

Can you make the target by concatenating together elements of the word bank

In [22]:
x = ["hello", "yo", "hi", "bye"]
x[:2] + x[3:]

['hello', 'yo', 'bye']

In [8]:
from copy import copy

In [51]:
def can_construct(
    target: Union[str, List[str],], 
    word_bank: List[str],
    memo: Optional[Dict[str, bool]] = None
)-> bool:
# ->Union[None, List[str]]:
  if memo is None:
    memo = {}
  if target in memo:
    return memo[target]
    
  # Base case
  if len(target) == 0:
    return True
  
  # Iterate
  for word in word_bank:
    # Look down tree if "word" is a prefix of target
    if word in target and target.index(word) == 0:
      construct_result =  can_construct(target[len(word):], word_bank, memo)
      if construct_result:
        memo[target] = True
        return True

  memo[target] = False
  return False


m is length of target string
n is length of word bank

Time Complexity: $O(n^m*m)$

Space Complexity: $O(m^2)$

Once memoied
Time complexity becomes $O(n*m^2)$

In [43]:
print_in(("skateboard", ["boa", "rd", "ate", "sk"]), can_construct)

('skateboard', ['boa', 'rd', 'ate', 'sk']):  True


In [53]:
print_in(("eeeeeeeeeeef", ["eee", "eeeeeee", "eeee"]), can_construct)

('eeeeeeeeeeef', ['eee', 'eeeeeee', 'eeee']):  False
